# SYCL devices

In [16]:
%%writefile compute.cpp
#include <iostream>
#include <sycl/sycl.hpp>
#include <vector>

int main() {
    const int N = 2;              // Length of the vectors
    std::vector<int> vectorA(N, 1);  // Vector A filled with 1s
    std::vector<int> vectorB(N, 2);  // Vector B filled with 2s
    int result = 0;  // Result of dot product initialized to zero

    auto myQueue = sycl::queue{sycl::gpu_selector_v};
    std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << std::endl;
{
    auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};
    auto bufB = sycl::buffer{vectorB.data(), sycl::range{N}};
    auto bufResult = sycl::buffer{&result, sycl::range{1}};

    myQueue.submit([&](sycl::handler& cgh) {
            auto accA = bufA.get_access<sycl::access::mode::read>(cgh);
            auto accB = bufB.get_access<sycl::access::mode::read>(cgh);
            auto accResult = bufResult.get_access<sycl::access::mode::read_write>(cgh);
            

            cgh.parallel_for<class vector_dot_product_gpu>(
                sycl::range{N}, [=](sycl::id<1> idx) {
                    accResult[0] += accA[idx] * accB[idx];
                });
        })
        .wait();
}
    std::cout << "Dot product completed. Result: " << result
              << std::endl;
    return 0;
}


Overwriting compute.cpp


In [17]:
%%writefile ./run-sycl.sh

#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [18]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

Running on Intel(R) Graphics [0x46a6]
Dot product completed. Result: 2


## Debug and print out

```c++
    myQueue.submit([&](sycl::handler& cgh) {
    sycl::stream sout(10240, 2560, cgh);

    cgh.parallel_for<class operation>(
        sycl::range{N}, [=](sycl::id<1> idx) {
        int i = idx[0];
        sout << "id global " << idx[0] << sycl::endl;
        }); //end parallel_for
    }) //end submit
    .wait();
```